In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

import warnings
import math

from tabulate import tabulate

# Our utilities
from Utils import *
from FinancialMetrics import *
from Momentum import *
from TitleOutOfMarket import *

## Settings

In [ ]:
warnings.filterwarnings('ignore')

## Constants
The CSV files must be in the same directory of the notebook

In [ ]:
ROLLING_WINDOW_SIZE = 180   # Number of days to consider in our regression
NUMBER_OF_TITLES = 10       # Number of titles inside the portfolio

CLOSING_PRICE_CSV = 'data/NASDAQ-100-CLOSING-PRICES.csv'
TBILL_CSV = 'data/13WEEKTBILLCOUPON.csv' 
CLOSING_PRICE_OF_REMOVED_CSV = 'data/CLOSING_PRICES_OF_REMOVED_TITLES.csv' 

## Data Pre-Processing
Loading the closing prices dataset. We then calculate log returns 

### Load the Closing Price Dataset
Historical closing prices (with titles swap)

In [ ]:

closing_prices = pd.read_csv(CLOSING_PRICE_CSV)
closing_prices.head()

### Calculate the log returns

In [ ]:
nasdaq100_returns = get_log_returns(CLOSING_PRICE_CSV)
nasdaq100_returns.fillna(np.nan, inplace=True)
nasdaq100_returns.head()
nasdaq100_returns.head().style.applymap(color_negative_red)

### Load the Closing Price Dataset of stock removed from the Nasdaq-100 Index
Historical prices of removed titles

In [ ]:

closing_prices_removed_titles = pd.read_csv(CLOSING_PRICE_OF_REMOVED_CSV)
closing_prices_removed_titles.head()

### Loading the Risk Free Asset (US Treasury Bill - 3 Months)
We've used the Coupon Equivalent. The Coupon Equivalent, also called the Bond Equivalent, or the Investment Yield, is the bill's yield based on the purchase price, discount, and a 365- or 366-day year. 

In [ ]:
tbill = pd.read_csv(TBILL_CSV)
tbill['DATE'] = pd.to_datetime(tbill['DATE'])
tbill = tbill.sort_values(by='DATE', ascending=True)
tbill = tbill.reset_index().drop(['index'], axis=1)
risk_free_rate = tbill['13 WEEKS COUPON EQUIVALENT'].mean()
print(f'Annual risk-free rate: {np.round(risk_free_rate, 2)}%')

## Rolling Linear Regression
We create a dataframe containing the ROLLING_WINDOW_SIZE (defult 6-month) rows of the returns dataframe based on a given starting day. In this dataframe will be removed all those titles that possibly entered or exited in that period (have nan values)

In [ ]:
def get_window_returns(days):
    window_returns = nasdaq100_returns.iloc[days: days + ROLLING_WINDOW_SIZE]
    window_returns = window_returns.reset_index().drop(['index'], axis=1)

    # Remove titles that are not in the Nasdaq-100 window range
    window_returns.dropna(axis=1, how='any', inplace=True)

    # Get the name of the columns
    titles = window_returns.columns.tolist()

    # Remove the first two element (Dates, NDX Index) because I don't need them
    titles = titles[2:]

    return window_returns, titles

## Example: First 180 days

In [ ]:
rolling_df, titles = get_window_returns(0)
print("In the first {} days, {} stocks will be taken from the index and analyzed.".format(ROLLING_WINDOW_SIZE, len(rolling_df.columns)))
rolling_df.head().style.applymap(color_negative_red)

# TITOLO GENERALE: TO REORDER
- returns calculator
- portfolio ranking
- portfolio builder

## Portfolio Implementation

AGGIUNGERE DESCRIZIONE

- TODO: RITORNARE ANCHE rank_df per fare dopo eventuali analisi nel report

In [ ]:
def portfolio_ranked(selector, days):
    """
    Build a portfolio, based on the selector, by taking the titles included in the index in the range [days; days + ROLLING_WINDOW_SIZE]
    :param selector: The selector to use in order to build the rank and select the titles
    :param selector_columns: Optional values to use in the specific selector
    :param days: The number of days to skip 
    """
    
    rolling_df, titles = get_window_returns(days)
    rank_df = pd.DataFrame(columns=['Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns', 'systematic_risk'])

    ndx_returns = rolling_df.iloc[:, 1].values

    for title in titles:
        title_returns = rolling_df.iloc[0 : ROLLING_WINDOW_SIZE, rolling_df.columns.get_loc(title)]

        ndx_returns = sm.add_constant(ndx_returns)
        model = sm.OLS(title_returns, ndx_returns)
        result = model.fit()

        rank_df = rank_df.append({'Title': title, 'r2': result.rsquared, 'specific_risk': np.var(result.resid), 'beta': result.params[1], 'alpha': result.params[0], 'alpha_significance': result.pvalues[0], 'absolute_returns': np.sum(title_returns),  'systematic_risk': result.params[1] ** 2 * np.var(ndx_returns)}, ignore_index=True)
        rank_df['total_risk'] = rank_df['systematic_risk'] + rank_df['specific_risk']

    full_rank_df = rank_df.copy()
    if selector == 'max_r2':
        winners = rank_df.sort_values(by='r2', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_total_risk':
        winners = rank_df.sort_values(by='total_risk', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_total_risk':
        winners = rank_df.sort_values(by='total_risk', ascending=True).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'excess_return_over_total':
        rank_df['excess_over_total'] = rank_df['alpha'] / rank_df['absolute_returns']
        winners = rank_df.sort_values(by='excess_over_total', ascending=True).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_specific_risk':
        winners = rank_df.sort_values(by='specific_risk', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'absolute_returns':
        winners = rank_df.sort_values(by='absolute_returns', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_beta':
        # The same as high_systematic_risk
        winners = rank_df.sort_values(by='beta', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_alpha':
        winners = rank_df.sort_values(by='alpha', ascending=True).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_r2_and_high_specific_risk':
        rank_df = rank_df.sort_values(by=['r2'], ascending=True)
        rank_df = rank_df.head(int(len(titles) * 1/3))
        winners = rank_df.sort_values(by='specific_risk', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_beta':
        winners = rank_df.sort_values(by='beta', ascending=True).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'high_systematic_risk' or selector == 'low_systematic_risk':
        ascending = False
        if 'low' in selector:
            ascending = True
        winners = rank_df.sort_values(by='systematic_risk', ascending=ascending).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_and_significant_alpha':
        rank_df = rank_df[rank_df['alpha_significance'] < 0.05]
        rank_df = rank_df[rank_df['alpha'] > 0]
        if(rank_df.shape[0] < NUMBER_OF_TITLES):
            raise Exception("Not enought titles")
        winners = rank_df.sort_values(by='alpha', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_alpha_and_high_beta':
        rank_df = rank_df[rank_df['alpha'] > 0]
        winners = rank_df.sort_values(by='beta', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    
    return selected_titles, winners, full_rank_df

In [ ]:
def get_weekly_portfolio_returns(window_returns):
    
    returns = []
    left_the_market = [] # Keep here title already out of the portfolio

    for index, row in window_returns.iterrows():

        portfolio_components_number = NUMBER_OF_TITLES
        daily_return = 0

        for title, value in row.items():
            if math.isnan(value):
                if title in left_the_market:
                    portfolio_components_number -= 1 # Skip this title. Removed from the market and from the portfolio
                else:
                    missing_date = nasdaq100_returns.iloc[index - 1]['Dates'] # Lo slicing di pandas sugli object ritorna l'indice + 1
                    missing_price = closing_prices_removed_titles.iloc[index][title]

                    if math.isnan(missing_price):
                        #print(f'{title} is Nan at row {index} ({missing_date}). It was removed')

                        if title in title_out_of_the_market.keys():
                            missing_price = title_out_of_the_market[title]
                            last_closing_price = closing_prices_removed_titles.iloc[index - 1][title]
                            missing_return = np.log(missing_price) - np.log(last_closing_price)
                            daily_return += missing_return
                        else:
                            raise Exception(f'Cannot find the closing/acquisition price of {title}')   

                        left_the_market.append(title)
                    else:
                        last_closing_price = closing_prices_removed_titles.iloc[index - 1][title]
                        missing_return = np.log(missing_price) - np.log(last_closing_price)
                        daily_return += missing_return
            else:
                 daily_return += value

        #if portfolio_components_number < NUMBER_OF_TITLES:
            #print(f'Last return: {returns[-1]}. New return: {daily_return / portfolio_components_number}. Number of titles {portfolio_components_number}')
            
        returns.append(daily_return / portfolio_components_number)
            
    return returns


In [ ]:
def portfolio_builder(selector):
    portfolio_returns = []
    portfolio = pd.DataFrame(columns=['Dates'] + [f'Title{i}' for i in range(1 , NUMBER_OF_TITLES + 1)] + ['Returns'])
    portfolio_history = pd.DataFrame(columns=['Dates', 'Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns'])

    titles, values, _ = portfolio_ranked(selector, 0)
    portfolio_history = portfolio_history.append(values, ignore_index=True)
       
    days_range = nasdaq100_returns.shape[0] - ROLLING_WINDOW_SIZE
    for days in range(7, days_range, 7):

        nasdaq100_window_returns = nasdaq100_returns[titles].iloc[ROLLING_WINDOW_SIZE + days - 7 : ROLLING_WINDOW_SIZE + days]

        tmp_returns = get_weekly_portfolio_returns(nasdaq100_window_returns)
        portfolio_returns = portfolio_returns + tmp_returns

        portfolio_row = {'Dates': nasdaq100_returns.iloc[ROLLING_WINDOW_SIZE + days]['Dates']}
        portfolio_row.update({f'Title{i}': titles[i - 1] for i in range(1, NUMBER_OF_TITLES + 1)})
        portfolio = portfolio.append(portfolio_row, ignore_index=True)
        titles, values, _ = portfolio_ranked(selector, days)
        portfolio_history = portfolio_history.append(values, ignore_index=True)

    portfolio_history = portfolio_history[:-NUMBER_OF_TITLES]
    dates = np.array(portfolio['Dates'])
    dates = np.repeat(dates, NUMBER_OF_TITLES)
    portfolio_history['Dates'] = dates
    return portfolio, portfolio_returns, portfolio_history
    

## Portfolio Selectors
- max_r2: desc..

DESCRIVERE I SELETTORI
E POI DARE UNA DESCRIZIONE A TUTTI I VARI DICTIONARY CHE CREIAMO

In [ ]:
selectors = [
    'max_r2',
    'absolute_returns',
    'min_r2_and_high_specific_risk', 
    'max_specific_risk',
    'high_systematic_risk',
    'low_systematic_risk',
    'min_beta', 
    'positive_alpha',
    'positive_and_significant_alpha',
    'positive_alpha_and_high_beta',
    'max_total_risk',
    'min_total_risk',
    'excess_return_over_total'
]

selectors = ['absolute_returns']

In [ ]:
base_metrics = pd.DataFrame(columns=['Portfolio Title', 'Annualized Returns', 'Annualized Volatility'])
advanced_metrics = pd.DataFrame(columns=['Portfolio Title', 'Sharpe Ratio', 'MDD', 'CL', 'Var 90', 'Var 95', 'Var 99', 'IR', 'M2'])
portfolios_selector_history = {}
portfolios_analysis = {}

In [ ]:
days_limit = 0
for i in range(7, nasdaq100_returns.shape[0] - 180, 7):
    days_limit = i
ndx_returns = nasdaq100_returns[180: days_limit + 180]['NDX Index'].tolist()

In [ ]:
for selector in selectors:
    try:
        print(f'Buildindg {selector} portfolio')        

        portfolio, returns, history = portfolio_builder(selector)
        
        portfolios_selector_history[selector] = history
        portfolios_analysis[selector] = returns

        basic_row = get_base_metrics(selector, returns)
        base_metrics = base_metrics.append(basic_row, ignore_index=True)
        advanced_row = get_advanced_metrics(selector, returns, ndx_returns, risk_free_rate)
        advanced_metrics = advanced_metrics.append(advanced_row, ignore_index=True)
        
    except Exception as e:
        print(f'Cannot build the portfolio for the selector {selector}: {str(e)}')


In [ ]:
# Adding the Nasdaq-100 to our tables
basic_row = get_base_metrics('Nasdaq-100', ndx_returns)
base_metrics = base_metrics.append(basic_row, ignore_index=True)

In [ ]:
print(tabulate(base_metrics, headers='keys', tablefmt='psql')) 

In [ ]:
print(tabulate(advanced_metrics, headers='keys', tablefmt='psql')) 

## Analysis of some results

#### Portfolio with the highest level of R-squared

In [ ]:
idx = 0 # Play with this
portfolios_selector_history['max_r2'].iloc[idx : idx + 10]

In [ ]:
mean = portfolios_selector_history['max_r2']['r2'].mean()
print(f'The average value of r-squared is: {np.round(mean, 4)}')

portfolios_selector_history['max_r2'].sort_values(by='r2', ascending=False)

#### Portfolio of stock with better absolute return

In [ ]:
idx = 0 # Play with this
portfolios_selector_history['absolute_returns'].iloc[idx : idx + 10]

In [ ]:
# First week
first_week_return = pd.DataFrame()
first_week_return['Date'] = nasdaq100_returns[180:187]['Dates']
first_week_return['Log Return'] = np.array(portfolios_analysis['absolute_returns'][idx : idx + 7])
first_week_return['Log Return'] = first_week_return['Log Return']
first_week_return

In [ ]:
titles = list(portfolios_selector_history['absolute_returns'][0:10]['Title'])
first_week_stocks_return = nasdaq100_returns[titles].iloc[ROLLING_WINDOW_SIZE + 7 - 7 : ROLLING_WINDOW_SIZE + 7]
first_week_stocks_return.style.applymap(color_negative_red)

# Visualizing Data

In [ ]:
def portfolios_comparison(money, portfolios):
    dates = np.array(nasdaq100_returns[181:]['Dates']) 
    ndx_returns = np.array(nasdaq100_returns[181:]['NDX Index'])
    
    returns = []
    dates_to_show = [dates[i] for i in np.linspace(0, len(dates) - 1, 20).astype(int)]

    returns.append(money * math.exp(ndx_returns[0]))
    for i in range(1, len(dates)):
        returns.append(returns[i-1] * math.exp(ndx_returns[i]))

    plt.figure(figsize=(20,10))
    plt.plot(dates, returns, label='NDX Index')

    for title, portfolio_returns in portfolios.items():
        returns = []
        returns.append(money * math.exp(portfolio_returns[0]))
        for i in range(1, len(dates)):
            returns.append(returns[i-1] * math.exp(portfolio_returns[i]))
        plt.plot(dates, returns, label=title)

    plt.xlabel("Date")
    plt.xticks(dates_to_show, rotation=45)
    plt.ylabel("Value ($)")
    plt.legend()
    plt.title("Portfolios Comparison")
    # Aggiungere legenda log return

In [ ]:
portfolios_comparison(1000, portfolios_analysis)

In [ ]:
ndx_investment_returns = cumulative_returns(np.array(ndx_returns)) * 1000


In [ ]:
sns.set(rc={'figure.figsize':(18,9)})
dates = np.array(nasdaq100_returns[ROLLING_WINDOW_SIZE: ROLLING_WINDOW_SIZE + days_limit]['Dates']) 
dates_to_show = [dates[i] for i in np.linspace(0, len(dates) - 1, 20).astype(int)]
l = sns.lineplot(x = dates, y = ndx_investment_returns, linewidth = 1.5)
l.yaxis.tick_right()
l.set_xticks(dates_to_show)
l.set_xticklabels(dates_to_show, rotation=45)
l.set_title('NDX')
l.legend

In [ ]:
maxr2_investment_returns = cumulative_returns(np.array(portfolios_analysis['max_r2'])) * 1000
sns.set(font="Verdana")
sns.set(rc={'figure.figsize':(18,9)})
dates = np.array(nasdaq100_returns[ROLLING_WINDOW_SIZE: ROLLING_WINDOW_SIZE + days_limit]['Dates']) 
dates_to_show = [dates[i] for i in np.linspace(0, len(dates) - 1, 20).astype(int)]
l = sns.lineplot(x = dates, y = ndx_investment_returns, linewidth = 1.5)
l1 = sns.lineplot(x = dates, y = maxr2_investment_returns, linewidth = 1.5)
l.yaxis.tick_right()
l.set_xticks(dates_to_show)
l.set_xticklabels(dates_to_show, rotation=45)
l.set_xlabel('Dates')
l.set_ylabel('Value of investment ($)')
l.set_title('NDX VS MAX R2 PORTFOLIO', size=14)
l.legend(labels=['NDX', 'MAX R2'], loc='upper left')

In [ ]:
maxr2_investment_returns = cumulative_returns(np.array(portfolios_analysis['max_specific_risk'])) * 1000
sns.set(font="Verdana")
sns.set(rc={'figure.figsize':(18,9)})
dates = np.array(nasdaq100_returns[ROLLING_WINDOW_SIZE: ROLLING_WINDOW_SIZE + days_limit]['Dates']) 
dates_to_show = [dates[i] for i in np.linspace(0, len(dates) - 1, 20).astype(int)]
l = sns.lineplot(x = dates, y = ndx_investment_returns, linewidth = 1.5, color='blue')
l1 = sns.lineplot(x = dates, y = maxr2_investment_returns, linewidth = 1.5, color='red')
l.yaxis.tick_right()
l.set_xticks(dates_to_show)
l.set_xticklabels(dates_to_show, rotation=45)
l.set_xlabel('Dates')
l.set_ylabel('Value of investment ($)')
l.set_title('NDX VS MAX SPECIFIC RISK PORTFOLIO', size=14)
l.legend(labels=['NDX', 'MAX SPECIFIC RISK'], loc='upper left')

# Non Compulsoty Tasks
- Momentum implmentation
- A different weights schema: Inverse Volatility
- Simple Returns vs Log Returns

## Momentum
Momentum strategy is a system of buying stocks or other securities that have had high returns over the past months. There are several ways to implement a momentum strategy. In this notebook we decide to implement the Clenow Momentum (by Andreas F. Clenow). We take 1/3 of the entire titles on the basis of momentum and then we redo the calculations done previously - but on a subset.

For more details:
- https://www.amazon.com/Stocks-Move-Beating-Momentum-Strategies/dp/1511466146
- https://www.quant-investing.com/blog/this-easy-to-use-adjusted-slope-momentum-strategy-performed-7-times-better-than-the-market

In [ ]:
# Reinitialization
base_metrics = pd.DataFrame(columns=['Portfolio Title', 'Annualized Returns', 'Annualized Volatility'])
advanced_metrics = pd.DataFrame(columns=['Portfolio Title', 'Sharpe Ratio', 'MDD', 'CL', 'Var 90', 'Var 95', 'Var 99', 'IR', 'M2'])
portfolios_selector_history = {}
portfolios_analysis = {}

### Core part

In [ ]:
def get_clenow_momentum(title, days):
    prices = closing_prices[title].iloc[days: days + ROLLING_WINDOW_SIZE]
    prices_log = np.log(prices)
    x = np.arange(len(prices_log)) 
    slope, _, rvalue, _, _ = linregress(x, prices_log)
    m = ((1 + slope) ** 252) * (rvalue ** 2)
    return m

In [ ]:
def get_window_returns_with_momentum(days):
    window_returns = nasdaq100_returns.iloc[days: days + ROLLING_WINDOW_SIZE]
    window_returns = window_returns.reset_index().drop(['index'], axis=1)

    window_returns.dropna(axis=1, how='any', inplace=True)
    titles = window_returns.columns.tolist()
    titles = titles[2:]

    momentums = pd.DataFrame(columns=['Title', 'Momentum'])
    momentums['Title'] = titles

    for index, tt in momentums.iterrows():
        momentums.at[index,'Momentum'] = get_clenow_momentum(tt.Title, days)

    # Get first 1/3 titles with the highest momentum
    momentums = momentums.sort_values(by='Momentum', ascending=False)
    momentums = momentums.iloc[:int(len(momentums) / 3)]
    momentums = momentums['Title'].tolist()


    return window_returns, momentums

In [ ]:
def portfolio_ranked_with_momentum(selector, days):

    rolling_df, titles = get_window_returns_with_momentum(days) 

    rank_df = pd.DataFrame(columns=['Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns', 'systematic_risk'])

    ndx_returns = rolling_df.iloc[:, 1].values

    for title in titles:
        title_returns = rolling_df.iloc[0 : ROLLING_WINDOW_SIZE, rolling_df.columns.get_loc(title)]

        ndx_returns = sm.add_constant(ndx_returns)
        model = sm.OLS(title_returns, ndx_returns)
        result = model.fit()

        rank_df = rank_df.append({'Title': title, 'r2': result.rsquared, 'specific_risk': np.var(result.resid), 'beta': result.params[1], 'alpha': result.params[0], 'alpha_significance': result.pvalues[0], 'absolute_returns': np.sum(title_returns),  'systematic_risk': result.params[1] ** 2 * np.var(ndx_returns)}, ignore_index=True)
        rank_df['total_risk'] = rank_df['systematic_risk'] + rank_df['specific_risk']

    full_rank_df = rank_df.copy()
    if selector == 'max_r2':
        winners = rank_df.sort_values(by='r2', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_total_risk':
        winners = rank_df.sort_values(by='total_risk', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_total_risk':
        winners = rank_df.sort_values(by='total_risk', ascending=True).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'excess_return_over_total':
        rank_df['excess_over_total'] = rank_df['alpha'] / rank_df['absolute_returns']
        winners = rank_df.sort_values(by='excess_over_total', ascending=True).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_specific_risk':
        winners = rank_df.sort_values(by='specific_risk', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'absolute_returns':
        winners = rank_df.sort_values(by='absolute_returns', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_beta':
        # The same as high_systematic_risk
        winners = rank_df.sort_values(by='beta', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_alpha':
        winners = rank_df.sort_values(by='alpha', ascending=True).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_r2_and_high_specific_risk':
        rank_df = rank_df.sort_values(by=['r2'], ascending=True)
        rank_df = rank_df.head(int(len(titles) * 1/3))
        winners = rank_df.sort_values(by='specific_risk', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_beta':
        winners = rank_df.sort_values(by='beta', ascending=True).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'high_systematic_risk' or selector == 'low_systematic_risk':
        ascending = False
        if 'low' in selector:
            ascending = True
        winners = rank_df.sort_values(by='systematic_risk', ascending=ascending).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_and_significant_alpha':
        rank_df = rank_df[rank_df['alpha_significance'] < 0.05]
        rank_df = rank_df[rank_df['alpha'] > 0]
        if(rank_df.shape[0] < NUMBER_OF_TITLES):
            raise Exception("Not enought titles")
        winners = rank_df.sort_values(by='alpha', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_alpha_and_high_beta':
        rank_df = rank_df[rank_df['alpha'] > 0]
        winners = rank_df.sort_values(by='beta', ascending=False).head(NUMBER_OF_TITLES)
        selected_titles = winners['Title'].tolist()
    
    return selected_titles, winners, full_rank_df

In [ ]:
def portfolio_builder_with_momentum(selector):
    portfolio_returns = []
    portfolio = pd.DataFrame(columns=['Dates'] + [f'Title{i}' for i in range(1,11)] + ['Returns'])
    portfolio_history = pd.DataFrame(columns=['Dates', 'Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns'])

    titles, values, _ = portfolio_ranked_with_momentum(selector, 0)
    portfolio_history = portfolio_history.append(values, ignore_index=True)
       
    days_range = nasdaq100_returns.shape[0] - ROLLING_WINDOW_SIZE
    for days in range(7, days_range, 7):

        nasdaq100_window_returns = nasdaq100_returns[titles].iloc[ROLLING_WINDOW_SIZE + days - 7 : ROLLING_WINDOW_SIZE + days]

        tmp_returns = get_weekly_portfolio_returns(nasdaq100_window_returns)
        portfolio_returns = portfolio_returns + tmp_returns

        portfolio_row = {'Dates': nasdaq100_returns.iloc[ROLLING_WINDOW_SIZE + days]['Dates']}
        portfolio_row.update({f'Title{i}': titles[i - 1] for i in range(1,11)})
        portfolio = portfolio.append(portfolio_row, ignore_index=True)

        titles, values, _ = portfolio_ranked_with_momentum(selector, days)
        portfolio_history = portfolio_history.append(values, ignore_index=True)

    portfolio_history = portfolio_history[:-10]
    dates = np.array(portfolio['Dates'])
    dates = np.repeat(dates, 10)
    portfolio_history['Dates'] = dates
    return portfolio, portfolio_returns, portfolio_history
    

In [ ]:
for selector in selectors:
    try:
        print(f'Buildindg {selector} portfolio')        
        portfolio, returns, history = portfolio_builder_with_momentum(selector)
        portfolios_selector_history[selector] = history
        portfolios_analysis[selector] = returns
        basic_row = get_base_metrics(selector, returns)
        base_metrics = base_metrics.append(basic_row, ignore_index=True)
        advanced_row = get_advanced_metrics(selector, returns, ndx_returns, risk_free_rate)
        advanced_metrics = advanced_metrics.append(advanced_row, ignore_index=True)
    except Exception as e:
        print(f'Cannot build the portfolio for the selector {selector}: {str(e)}')


In [ ]:
print(tabulate(base_metrics, headers='keys', tablefmt='psql')) 
print(tabulate(advanced_metrics, headers='keys', tablefmt='psql')) 

In [ ]:
print(tabulate(advanced_metrics, headers='keys', tablefmt='psql')) 

## New Weights Schema
In inverse volatility strategy the risk is measured with volatility, and assets are weighted in inverse proportion to their risk. An inverse volatility weighted portfolio is one in which highly volatile assets are assigned smaller weights and low volatile assets are allotted larger weights.

In [ ]:
# Reinitialization
base_metrics = pd.DataFrame(columns=['Portfolio Title', 'Annualized Returns', 'Annualized Volatility'])
advanced_metrics = pd.DataFrame(columns=['Portfolio Title', 'Sharpe Ratio', 'MDD', 'CL', 'Var 90', 'Var 95', 'Var 99', 'IR', 'M2'])
portfolios_selector_history = {}
portfolios_analysis = {}

In [ ]:
def get_weekly_portfolio_returns_inverse_volatility(window_returns, days):
    
    returns = []
    left_the_market = []
    title_volatility = {}

    # Get past returns - decide a priori weights
    titles = list(window_returns.columns)
    nasdaq100_past_returns = nasdaq100_returns[titles].iloc[days - 7 : ROLLING_WINDOW_SIZE + days - 7]
    for title in titles:
        title_returns = nasdaq100_past_returns[title] 
        title_volatility[title] = 1 / title_returns.std()

    for index, row in window_returns.iterrows():
        daily_return = 0
        total_volatility = 0

        # Get total volatility
        for title, value in row.items():
            if math.isnan(value):
                if title in left_the_market:
                    pass
                else:
                    missing_price = closing_prices_removed_titles.iloc[index][title]
                    if math.isnan(missing_price):
                        if title in title_out_of_the_market.keys():
                            total_volatility += title_volatility[title]
                        else:
                            raise Exception(f'Cannot find the closing/acquisition price of {title}')   
                        left_the_market.append(title)
                    else:
                        total_volatility += title_volatility[title]
            else:
                 total_volatility += title_volatility[title]

        for title, value in row.items():
            if math.isnan(value):
                if title in left_the_market:
                    pass
                else:
                    missing_price = closing_prices_removed_titles.iloc[index][title]
                    if math.isnan(missing_price):
                        missing_price = title_out_of_the_market[title]
                        last_closing_price = closing_prices_removed_titles.iloc[index - 1][title]
                        missing_return = np.log(missing_price) - np.log(last_closing_price)
                        daily_return += missing_return * (title_volatility[title] / total_volatility)
                        left_the_market.append(title)
                    else:
                        last_closing_price = closing_prices_removed_titles.iloc[index - 1][title]
                        missing_return = np.log(missing_price) - np.log(last_closing_price)
                        daily_return += missing_return * (title_volatility[title] / total_volatility)
            else:
                 daily_return += value * (title_volatility[title] / total_volatility)

        returns.append(daily_return)
            
    return returns


In [ ]:
def portfolio_builder_inverse_volatility(selector):
    portfolio_returns = []
    portfolio = pd.DataFrame(columns=['Dates'] + [f'Title{i}' for i in range(1 , NUMBER_OF_TITLES + 1)] + ['Returns'])
    portfolio_history = pd.DataFrame(columns=['Dates', 'Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns'])

    titles, values, _ = portfolio_ranked(selector, 0)
    portfolio_history = portfolio_history.append(values, ignore_index=True)
       
    days_range = nasdaq100_returns.shape[0] - ROLLING_WINDOW_SIZE
    for days in range(7, days_range, 7):

        nasdaq100_window_returns = nasdaq100_returns[titles].iloc[ROLLING_WINDOW_SIZE + days - 7 : ROLLING_WINDOW_SIZE + days]

        tmp_returns = get_weekly_portfolio_returns_inverse_volatility(nasdaq100_window_returns, days)
        portfolio_returns = portfolio_returns + tmp_returns

        portfolio_row = {'Dates': nasdaq100_returns.iloc[ROLLING_WINDOW_SIZE + days]['Dates']}
        portfolio_row.update({f'Title{i}': titles[i - 1] for i in range(1, NUMBER_OF_TITLES + 1)})
        portfolio = portfolio.append(portfolio_row, ignore_index=True)
        titles, values, _ = portfolio_ranked(selector, days)
        portfolio_history = portfolio_history.append(values, ignore_index=True)

    portfolio_history = portfolio_history[:-NUMBER_OF_TITLES]
    dates = np.array(portfolio['Dates'])
    dates = np.repeat(dates, NUMBER_OF_TITLES)
    portfolio_history['Dates'] = dates
    return portfolio, portfolio_returns, portfolio_history
    

In [ ]:
for selector in selectors:
    try:
        print(f'Buildindg {selector} portfolio')        

        portfolio, returns, history = portfolio_builder_inverse_volatility(selector)
        
        portfolios_selector_history[selector] = history
        portfolios_analysis[selector] = returns

        basic_row = get_base_metrics(selector, returns)
        base_metrics = base_metrics.append(basic_row, ignore_index=True)
        advanced_row = get_advanced_metrics(selector, returns, ndx_returns, risk_free_rate)
        advanced_metrics = advanced_metrics.append(advanced_row, ignore_index=True)
        
    except Exception as e:
        print(f'Cannot build the portfolio for the selector {selector}: {str(e)}')


In [ ]:
print(tabulate(base_metrics, headers='keys', tablefmt='psql')) 
print(tabulate(advanced_metrics, headers='keys', tablefmt='psql')) 

## Simple return vs Log returns
Here we transform log returns to simple returns and see how portfolios returns change

In [ ]:
# Reinitialization
base_metrics = pd.DataFrame(columns=['Portfolio Title', 'Annualized Returns', 'Annualized Volatility'])
advanced_metrics = pd.DataFrame(columns=['Portfolio Title', 'Sharpe Ratio', 'MDD', 'CL', 'Var 90', 'Var 95', 'Var 99', 'IR', 'M2'])
portfolios_selector_history = {}
portfolios_analysis = {}

In [ ]:
def get_weekly_portfolio_returns_with_simple_returns(window_returns):
    
    #Apply the conversion
    window_returns = np.exp(window_returns) - 1

    returns = []
    left_the_market = []
    for index, row in window_returns.iterrows():
        portfolio_components_number = NUMBER_OF_TITLES
        daily_return = 0
        for title, value in row.items():
            if math.isnan(value):
                if title in left_the_market:
                    portfolio_components_number -= 1
                else:
                    missing_price = closing_prices_removed_titles.iloc[index][title]
                    if math.isnan(missing_price):
                        if title in title_out_of_the_market.keys():
                            missing_price = title_out_of_the_market[title]
                            last_closing_price = closing_prices_removed_titles.iloc[index - 1][title]
                            missing_return = (missing_price - last_closing_price) / last_closing_price
                            daily_return += missing_return
                        else:
                            raise Exception(f'Cannot find the closing/acquisition price of {title}')   
                        left_the_market.append(title)
                    else:
                        last_closing_price = closing_prices_removed_titles.iloc[index - 1][title]
                        missing_return = (missing_price - last_closing_price) / last_closing_price
                        daily_return += missing_return
            else:
                 daily_return += value
        returns.append(daily_return / portfolio_components_number)  
    return returns


In [ ]:
def portfolio_builder_with_simple_returns(selector):
    portfolio_returns = []
    portfolio = pd.DataFrame(columns=['Dates'] + [f'Title{i}' for i in range(1 , NUMBER_OF_TITLES + 1)] + ['Returns'])
    portfolio_history = pd.DataFrame(columns=['Dates', 'Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns'])

    titles, values, _ = portfolio_ranked(selector, 0)
    portfolio_history = portfolio_history.append(values, ignore_index=True)
       
    days_range = nasdaq100_returns.shape[0] - ROLLING_WINDOW_SIZE
    for days in range(7, days_range, 7):

        nasdaq100_window_returns = nasdaq100_returns[titles].iloc[ROLLING_WINDOW_SIZE + days - 7 : ROLLING_WINDOW_SIZE + days]

        tmp_returns = get_weekly_portfolio_returns_with_simple_returns(nasdaq100_window_returns)
        portfolio_returns = portfolio_returns + tmp_returns

        portfolio_row = {'Dates': nasdaq100_returns.iloc[ROLLING_WINDOW_SIZE + days]['Dates']}
        portfolio_row.update({f'Title{i}': titles[i - 1] for i in range(1, NUMBER_OF_TITLES + 1)})
        portfolio = portfolio.append(portfolio_row, ignore_index=True)
        titles, values, _ = portfolio_ranked(selector, days)
        portfolio_history = portfolio_history.append(values, ignore_index=True)

    portfolio_history = portfolio_history[:-NUMBER_OF_TITLES]
    dates = np.array(portfolio['Dates'])
    dates = np.repeat(dates, NUMBER_OF_TITLES)
    portfolio_history['Dates'] = dates
    return portfolio, portfolio_returns, portfolio_history
    

In [ ]:
for selector in selectors:
    try:
        print(f'Buildindg {selector} portfolio')        

        portfolio, returns, history = portfolio_builder_with_simple_returns(selector)
        
        portfolios_selector_history[selector] = history
        portfolios_analysis[selector] = returns

        basic_row = get_base_metrics(selector, returns)
        base_metrics = base_metrics.append(basic_row, ignore_index=True)
        advanced_row = get_advanced_metrics(selector, returns, ndx_returns, risk_free_rate)
        advanced_metrics = advanced_metrics.append(advanced_row, ignore_index=True)
        
    except Exception as e:
        print(f'Cannot build the portfolio for the selector {selector}: {str(e)}')


In [ ]:
# Adding the Nasdaq-100 to our tables
ndx_simple_returns = np.exp(ndx_returns) - 1
basic_row = get_base_metrics('Nasdaq-100', ndx_simple_returns)
base_metrics = base_metrics.append(basic_row, ignore_index=True)

In [ ]:
print(tabulate(base_metrics, headers='keys', tablefmt='psql')) 
print(tabulate(advanced_metrics, headers='keys', tablefmt='psql')) 